In [107]:
import pytest
import numpy as np

**Таблица сопряжённости** (матрица неточности) содержит сводные показатели качества работы классификатора.
* **Строки** соответствуют **фактическим** классам тестового набора;
* **Cтолбцы** соответствуют **предсказанным** классом.

Таблица содержит четыре сводных показателя, каждый из которых отражает количество объектов в одной и четырех
категорий:
* **Истинно позитивный** (*True positive*, **TP**) -- объект
класса `1` был верно помечен меткой `1`;
* **Ложно позитивный** (*False positive*, **FP**) -- объект
фактически принадлежит классу `0`, но помечен меткой `1`;
* **Истинно отрицательный** (*True negative*, **TN**) -- классификатор
верно определил, что объект класса `0` принадлежит классу `0`;
* **Ложно отрицательный** (*False negative*, **FN**) -- классификатор
пометил объект меткой `0`, однако на самом деле объект принадлежит классу `1`.


|                    | Предсказано `0` | Предсказано `1` |
|:-------------------|:----------------|:----------------|
| **Фактически** `0` | TN              | FP              |
| **Фактически** `1` | FN              | TP              |


In [108]:
class ConfusionMatrix:
    def __init__(self, y_true=None, y_pred=None):
        self._confusion_matrix = np.array([[0,0], [0,0]])
        if not y_true is None and not y_pred is None:
            self.upd(y_true, y_pred)

    def upd(self, y_true, y_pred):
        for value, prediction in zip(y_true, y_pred):
            self._confusion_matrix[value][prediction] += 1

    def get(self):
        return self._confusion_matrix

    def get_flatten(self):
        return self._confusion_matrix.ravel()

In [109]:
def f1_score(y_true, y_pred):
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    return tp / (tp + (fp + fn) / 2)

def accuracy_score(y_true, y_pred):
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    return (tn + tp) / (tn + fp + fn + tp)

def precision_score(y_true, y_pred):
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    return tp / (tp + fp)

def recall_score(y_true, y_pred):
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    return tp / (fn + tp)

In [110]:
#import matplotlib.pyplot as plt

In [111]:
def roc_curve(y_true, y_proba, threshold):
    y_pred = np.where(y_proba >= threshold, 1, 0)
    confusion_matrix = ConfusionMatrix(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix.get_flatten()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    return fpr, tpr, threshold

def roc_auc(y_true, y_proba):
    thresholds = np.linspace(0, 1, 100)
    auc = 0
    for th in thresholds:
        fpr, tpr, th = roc_curve(y_true, y_proba, th)
        auc = max((1 + tpr - fpr) / 2, auc)
    return auc

In [112]:
import sklearn.metrics as metrics
import math

In [113]:
def test_computations(y_true, y_pred):
    assert math.isclose(f1_score(y_true, y_pred), metrics.f1_score(y_true, y_pred))
    assert math.isclose(accuracy_score(y_true, y_pred), metrics.accuracy_score(y_true, y_pred))
    assert math.isclose(precision_score(y_true, y_pred), metrics.precision_score(y_true, y_pred))
    assert math.isclose(recall_score(y_true, y_pred), metrics.recall_score(y_true, y_pred))

In [114]:
import random

In [115]:
y_true = np.array([0,1,1,0,0,1,1,0,0,0])
y_pred = np.array([0,1,1,0,0,1,0,1,0,1])
y_proba = np.array([(random.randint(0, len(y_true)) / len(y_true)) for i in range(len(y_true))])
test_computations(y_true, y_pred)

In [116]:
# print(roc_auc(y_true, y_proba))
# print(metrics.roc_auc_score(y_true,y_proba))

0.75
0.75
